In [19]:
import numpy as np

data = np.load(r"/home/foresti/Score-Entropy-Discrete-Diffusion/genomic-FM/train_clinvar.npy")
data = data[:10000]

In [ ]:
print(data.shape)
print(np.unique(data[:,-1], return_counts=True))
class_labels = np.unique(data[:,-1], return_counts=True)[1]
p_class_labels = class_labels/sum(class_labels)
print(p_class_labels)

(10000, 33)
(array([0, 1, 2, 3]), array([1509, 7650,  395,  446]))
